In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
from torch import distributions as dist

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
iris = datasets.load_iris()
X = iris['data']
y = iris['target']
X.shape, y.shape, set(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
ds_train = TensorDataset(
    torch.Tensor(X_train), 
    torch.Tensor(y_train).long()
)

ds_test = TensorDataset(
    torch.Tensor(X_test), 
    torch.Tensor(y_test).long()
)

batch_size=16
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_test = DataLoader(ds_test, batch_size, shuffle=True)

In [ ]:
model = nn.Sequential(
    nn.Linear(4, 10),
#     nn.ReLU(),
    nn.Linear(10, 20),
# #     nn.ReLU(),
    nn.Linear(20, 3)
)

opt = optim.Adam(model.parameters(), 4e-3)
criterion = nn.CrossEntropyLoss()
number_of_epochs=40
model

In [ ]:
for i in range(number_of_epochs):
    train_loss = 0
    model = model.train()
    y_train_predicted = []
    y_train_true = []
    for iteration, sample in enumerate(dl_train):
        imgs, y = sample

        model.zero_grad()
        output = model(imgs)
        loss = criterion(output, y)
        train_loss += loss.item() / imgs.size(0)
        loss.backward()
        opt.step()
        _, pred = torch.max(output, dim=1)
        y_train_predicted.extend(pred.detach().cpu().tolist())
        y_train_true.extend(y.detach().cpu().tolist())
        
    val_loss = 0
    y_predicted = []
    y_true = []

    with torch.no_grad():
        model = model.eval()
        for iteration, sample in enumerate(dl_test):
            imgs, y = sample

            output = model(imgs)
            loss = criterion(output, y)
            val_loss += loss.item() / imgs.size(0)
            _, pred = torch.max(output, dim=1)
            y_predicted.extend(pred.cpu().tolist())
            y_true.extend(y.cpu().tolist())
            
    val_acc = accuracy_score(y_true, y_predicted)
    print('#Epoch: {}, train loss: {}, test loss: {}, val_acc: {}'.format(i, train_loss, val_loss, val_acc))
    

# Gaussian estimator

In [ ]:
true_d = dist.multivariate_normal.MultivariateNormal(torch.Tensor([1,1]), torch.Tensor([[1,0],[0,1]]))
X = true_d.sample((100,))
des_loss = - true_d.log_prob(X)

print("desired loss:", des_loss.mean())
mean_v = torch.autograd.Variable(torch.Tensor(np.random.rand(2)), requires_grad=True)
cov_v = torch.autograd.Variable(torch.Tensor(np.diag(np.abs(np.random.rand(2)))), requires_grad=True)

params = [mean_v, cov_v]


opt = optim.Adam(params, 0.01)

for i in range(100):
    opt.zero_grad()
    d = dist.multivariate_normal.MultivariateNormal(mean_v, cov_v)

    loss = - d.log_prob(X).mean()
    loss.backward()
    opt.step()
    for p in params:
        p.grad.data.zero_()

print("trained loss:", loss)

In [ ]:
P = torch.rand(10)

P_var = torch.autograd.Variable(P, requires_grad=True)
print("pre", P_var)
opt = optim.Adam([P_var], 0.01)

for i in range(100):
    opt.zero_grad()
    loss = - torch.log(
        torch.nn.functional.softmax(P_var)
    ).sum()
    print(loss)
    loss.backward()
    opt.step()
    P_var.grad.data.zero_()

print("post", P_var)

# MNIST

In [ ]:
transforms = tr.Compose([])

In [ ]:
ds = MNIST('./data', train=True, target_transform=None, download=True, transform=transforms)
ds_test = MNIST('./data', train=False, target_transform=None, download=True, transform=transforms)

batch_size = 32
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

In [ ]:
i, l = ds[0]
i